In [ ]:
from konlpy.tag import Okt
okt = Okt()

from tqdm import tqdm
import torch
import torch.nn as nn

import torch
import torch.nn.functional as F

from konlpy.tag import Okt
import re

import pandas as pd
import numpy as np
import pickle

In [ ]:
# 레시피 데이터 불러오기
import pickle
with open('recipe_step_dict.pkl', 'rb') as f:
    df1 = pickle.load(f)
    
# 식재료 벡터 불러오기
import pickle
with open('ingre.pickle', 'rb') as f:
    test = pickle.load(f)
    
# 조리방법 사전 불러오기
import pickle
with open('unique_stems.pkl', 'rb') as f:
    unique_stems = pickle.load(f)

In [ ]:
# 레시피 데이터 가지고 데이터프레임 생성
df2 = pd.DataFrame(df1.values(), columns=['recipe_step'])

#recipe_step이 None인 행 삭제
df2 = df2.dropna(subset=['recipe_step'])

# 삭제 후 남는 행 확인
df2[df2['recipe_step'].isnull()]

In [ ]:
# 'recipe_step' 열 값이 빈 리스트인 행을 제거
df2 = df2[df2['recipe_step'] != "[]"]

# 인덱스 초기화
df2 = df2.reset_index(drop=True)
recipe_step = df2['recipe_step']

In [ ]:
# 활용할 레시피 개수 선택
recipe_step_30 = recipe_step[:10]

In [ ]:
# 레시피 데이터에 있는 모든 동사 추출
stems = []
for recipe_step in tqdm(recipe_step_30):
    pos_list = okt.pos(recipe_step)
    verbs = [word for word, pos in pos_list if pos.startswith('Verb')]
    for verb in verbs:
        stem = okt.morphs(verb, stem = True)
        stems.append(stem[0])

In [ ]:
# Okt 형태소 분석기 초기화
okt = Okt()

# 동사와 명사 추출 후 동사는 어간으로 변환하는 함수
def extract_verb_noun(text, food_vec):
    vn = []
    # 특수문자 및 숫자 제거
    text = re.sub('[^가-힣\s]', '', text)
    # 형태소 분석
    morphs = okt.pos(text)
    # 명사와 동사 추출

    for word, pos in morphs:
        if pos.startswith('N') and word in test:  # 명사일 경우
            vn.append(word)
        elif pos.startswith('V'):  # 동사일 경우
            # 동사 그대로 저장
            verb_stem = okt.pos(word, stem=True)[0][0]
            if verb_stem in unique_stems:
                vn.append(verb_stem)
    return vn

# 조리방법(동사), 식재료(명사) 저장할 리스트 생성
verb_noun_list = []

# 데이터에 대해 함수 적용하여 리스트에 추가
for text in tqdm(recipe_step_30):
    verb_noun = extract_verb_noun(text, test)
    verb_noun_list.append(verb_noun)

In [ ]:
# 레시피별 스텝을 5개 토큰으로 슬라이스
slice_num = 5
sliced_verb_noun = []
for verb_noun in verb_noun_list:
    # 5보다 적은 토큰을 가지고 있는 레시피는 ''를 채워서 5로 맞춰줌
    if len(verb_noun) < 5:
        for i in range(5-len(verb_noun)):
            verb_noun.append('')
    # 5개 이상의 레시피를 가진 레시피는 5개의 토큰으로 슬라이싱
    split_lists = [verb_noun[i:i+slice_num] for i in range(len(verb_noun)-slice_num+1)]
    sliced_verb_noun.append(split_lists)

In [ ]:
# 레시피 별로 5개 토큰으로 슬라이싱된 리스트 합치기
# target 값인 y_list 생성
sliced_verb_noun_total = []
y_list = []
for i, recipe in enumerate(sliced_verb_noun):
    for step in recipe:
        sliced_verb_noun_total.append(step)
        y_list.append(i)

In [ ]:
# 100차원의 0으로 채워진 배열 생성
dim = 100
zero_array = np.zeros(dim)

In [ ]:
# 단어 사전의 크기와 임베딩 차원을 정의
vocab_size = len(unique_stems)  # 단어 사전의 크기
embedding_dim = 100  # 임베딩 차원

# 임베딩 레이어 초기화
embedding = nn.Embedding(vocab_size, embedding_dim)

# 레시피 별로 슬라이스 된 데이터를 조리방법은 임베딩 레이어를 통과시켜 100차원으로 변경하고 식재료는 미리 만들어둔 100차원 벡터로 변경
recipe_vec = []
for i, sliced_recipe in enumerate(tqdm(sliced_verb_noun_total)):
    sliced_vec = []
    for token in sliced_recipe:
        if token in unique_stems:
            index = unique_stems.index(token)
            word_index = torch.LongTensor([index])
            word_embed = embedding(word_index)
            word_embed = word_embed.squeeze(0)
            sliced_vec.append(word_embed.detach().numpy())
            
        elif token in test:
            food_token = test[token]
            sliced_vec.append(food_token)
            
        elif token == '':
            sliced_vec.append(zero_array)

    recipe_vec.append(sliced_vec)        

In [ ]:
# input data, output data 텐서로 변경
recipe_tensor = torch.tensor(recipe_vec)
y_tensor = torch.tensor(y_list)

In [ ]:
# 데이터셋 생성
dataset = torch.utils.data.TensorDataset(recipe_tensor, y_tensor)

# 데이터로더 생성 (배치 활용)
batch_size = 100  # 배치 크기
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

### 하이퍼파라미터 튜닝 (그리드 서치)

In [ ]:
learning_rates = [0.001, 0.01, 0.1]  # 학습률에 대한 후보 값들
hidden_sizes = [64, 128, 256] # hidden_size(은닉츠)에 대한 후보 값들
best_loss_avg = 100 # 스타트 포인트
epochs = 200


for learning_rate in learning_rates:
    for hidden in hidden_sizes:
        # 하이퍼파라미터 설정
        input_size_noun = 100 # 입력 크기
        hidden_size_noun = hidden  # 은닉 상태 크기
        output_size_noun = len(recipe_step_30)  # 출력 크기

        # 식재료 RNN 모델 정의
        class RNNNoun(nn.Module):
            def __init__(self, input_size, hidden_size, output_size):
                super(RNNNoun, self).__init__()
                self.hidden_size = hidden_size # 은닉 상태 크기
                self.dense = nn.Linear(input_size, hidden_size)  # Dense layer를 통해 input_size에서 hidden_size로 크기 선형 변환
                self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True) # RNN층
                self.fc = nn.Linear(hidden_size, output_size) # fully connected층(레시피 개수의 크기로 사이즈 변환)
                self.softmax = nn.LogSoftmax(dim=1) # 소프트맥스 활성화 함수

            def forward(self, x):
                x = self.dense(x)
                h0 = torch.zeros(1, x.size(0), self.hidden_size) # 초기 은닉 상태 생성
                out, _ = self.rnn(x, h0) # RNN층 실행
                out1 = out[:, -1, :] # 마지막 RNN셀의 출력 선택(RNN 다대일 조건 추가)
                out2 = self.fc(out1) # fully conncected층
                out3 = self.softmax(out2) # 소프트맥스 층
                return out1, out2, out3

        # 모델 초기화
        model_noun = RNNNoun(input_size_noun, hidden_size_noun, output_size_noun)
        model_noun = model_noun.float() 

        # 손실 함수와 옵티마이저 설정
        criterion = nn.CrossEntropyLoss()
        optimizer_noun = torch.optim.Adam(model_noun.parameters(), lr=learning_rate)

        loss_list=[]
        for epoch in range(epochs):
            for batch in dataloader:  # 데이터로더에서 미니배치를 가져옴
                inputs, targets = batch  # 입력과 타겟 로드
                inputs = inputs.float()  # 입력 데이터 형식 변환

                optimizer_noun.zero_grad()  # 옵티마이저 초기화
                output1, output2, output3 = model_noun(inputs.squeeze(-1))  # 모델에 입력
                loss = criterion(output3, targets)  # 손실 계산
                loss.backward()  # 역전파를 통한 그래디언트 계산
                optimizer_noun.step()  # 옵티마이저로 모델의 가중치 업데이트
                loss_list.append(loss)

            if (epoch+1) % 10 == 0:  # 10번의 에폭마다 손실 출력
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

        loss_avg = average = sum(loss_list) / len(loss_list)
        print(f"Learning Rate: {learning_rate}")
        print(f"Best Hidden Size: {hidden}")
        print(f"평균 loss: {loss.item():.4f}")
        print()

        if loss_avg < best_loss_avg:
            best_loss_avg = loss_avg
            best_lr = learning_rate
            best_hidden_size = hidden
            
print(f"Best_loss_avg: {best_loss_avg.item():.4f}")
print(f"Best Learning Rate: {best_lr}")
print(f"Best Hidden Size: {best_hidden_size}")

In [ ]:
# 식재료 RNN 모델 정의
class RNNNoun(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNNoun, self).__init__()
        self.hidden_size = hidden_size # 은닉 상태 크기
        self.dense = nn.Linear(input_size, hidden_size)  # Dense layer를 통해 input_size에서 hidden_size로 크기 선형 변환
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True) # RNN층
        self.fc = nn.Linear(hidden_size, output_size) # fully connected층(레시피 개수의 크기로 사이즈 변환)
        self.softmax = nn.LogSoftmax(dim=1) # 소프트맥스 활성화 함수

    def forward(self, x):
        x = self.dense(x)
        h0 = torch.zeros(1, x.size(0), self.hidden_size) # 초기 은닉 상태 생성
        out, _ = self.rnn(x, h0) # RNN층 실행
        out1 = out[:, -1, :] # 마지막 RNN셀의 출력 선택(RNN 다대일 조건 추가)
        out2 = self.fc(out1) # fully conncected층
        out3 = self.softmax(out2) # 소프트맥스 층
        return out1, out2, out3
            
# 하이퍼파라미터 설정
input_size_noun = 100 # 입력 크기
hidden_size_noun = 256  # 은닉 상태 크기
output_size_noun = len(recipe_step_30)  # 출력 크기

# 모델 초기화
model_noun = RNNNoun(input_size_noun, hidden_size_noun, output_size_noun)
model_noun = model_noun.float() 

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer_noun = torch.optim.Adam(model_noun.parameters(), lr=0.001)

In [ ]:
loss_list=[]
epochs = 200
for epoch in range(epochs):
    for batch in dataloader:  # 데이터로더에서 미니배치를 가져옴
        inputs, targets = batch  # 입력과 타겟 로드
        inputs = inputs.float()  # 입력 데이터 형식 변환

        optimizer_noun.zero_grad()  # 옵티마이저 초기화
        output1, output2, output3 = model_noun(inputs.squeeze(-1))  # 모델에 입력
        loss = criterion(output3, targets)  # 손실 계산
        loss.backward()  # 역전파를 통한 그래디언트 계산
        optimizer_noun.step()  # 옵티마이저로 모델의 가중치 업데이트
        loss_list.append(loss)

    if (epoch+1) % 10 == 0:  # 10번의 에폭마다 손실 출력
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')